In [1]:
import os
import numpy as np
import pandas as pd
from functools import reduce

from ipywidgets import interact
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Legend, Select
from bokeh.plotting import figure
from bokeh.themes import Theme
import bokeh.plotting as bp
from bokeh.io import show, output_notebook, push_notebook

output_notebook()

Loading BokehJS ...

In [2]:
simulation_data_folder = os.path.join('../tests/data/simulations')
dct_simulation_labels = {'jcoin_random': 'Complete Randomization',
                         'jcoin_urn': 'β=1, D=χ2', 
                         'd_range': 'β=1, D=range', 
                         'b2': 'β=2, D=χ2', 
                         'jcoin_custom_d': 'Access app; β=1, D=χ2'}
lst_colors = ['red', 'blue', 'green', 'cyan', 'violet']
lst_quantiles = [x/10 for x in range(1, 11)]
lst_col = ['n_participants', 'd_min', 'd_max', 'max_d (mean)', 'd_min_ci', 'd_max_ci'] + \
          ['max_d ({0}th_pctl)'.format(int(thres * 100)) for thres in lst_quantiles] +  \
          ['max_asgmt_prop_diff_ge_{0}% (%)'.format(int(thres * 100)) for thres in lst_quantiles]

In [3]:
lst_df_simulation_plot_stats = [pd.read_csv(
    os.path.join(simulation_data_folder, simulation_label, 'trial_level_agg_stats.csv'))[lst_col]
                                .rename(columns=dict([(col, col + ' ({0})'.format(dct_simulation_labels[simulation_label]))
                                                        for col in lst_col if col != 'n_participants']))
                                for simulation_label in dct_simulation_labels.keys()]

In [4]:
pdf_plot_data = reduce(lambda left, right: left.merge(right,
                                                      on='n_participants',
                                                      how='outer'
                                                      ),
                       tuple(lst_df_simulation_plot_stats))
pdf_plot_data = pdf_plot_data.loc[pdf_plot_data['n_participants'] <= 2500]

In [5]:
def get_data_to_display(fname, col_suffix, thres):
    lst_case_labels = sorted([dct_simulation_labels[simulation_label] for simulation_label in dct_simulation_labels.keys()])
    lst_col = [col_suffix.format(thres, dct_simulation_labels[simulation_label]) for simulation_label in dct_simulation_labels.keys()
                                 ]
    pdf_data_to_display = pdf_plot_data[['n_participants'] + lst_col]
    pdf_data_to_display = pdf_data_to_display.rename(columns=dict([(col, [case for case in lst_case_labels if case in col][0])
                                                                   for col in lst_col if col != 'n_participants']))
    return pdf_data_to_display

In [12]:
def bkapp(doc):
    thres = 50
    p = figure(title="Maximum difference in assignment proportions at different thresholds", 
           plot_height=600, plot_width=1000, y_range=(-1,105), x_range=[0, 200],
           background_fill_color='#efefef')
    p.xaxis.axis_label = 'No. participants'
    p.xaxis.axis_label = '% of trials with Maximum difference in assignment proportions >= threshold'
    pdf_data_to_display = get_data_to_display('trial_level_agg_stats.csv', 'max_asgmt_prop_diff_ge_{0}% (%) ({1})', thres)
    trial_all_plot_source = bp.ColumnDataSource(pdf_data_to_display)
    lst_y_col = [dct_simulation_labels[simulation_label] for simulation_label in dct_simulation_labels.keys()]
    lst_line_plots = []
    p.add_layout(Legend(), 'right')
    colr = lst_colors[1]
    col = lst_y_col[1]
    for (colr, leg) in zip(lst_colors, lst_y_col):
        p.line('n_participants', leg, color=colr, legend_label=leg, source=trial_all_plot_source, #name='wave',
            nonselection_alpha=0.4, selection_alpha=1)
    
    thres_input = Select(value=90, title='Select a threshold', options=list(range(10, 110, 10)))

    def update_threshold(attr, old, new):
        pdf_data_to_display = get_data_to_display('trial_level_agg_stats.csv', 'max_asgmt_prop_diff_ge_{0}% (%) ({1})', thres)
        trial_all_plot_source.data.update(bp.ColumnDataSource(pdf_data_to_display).data)
    
    file_picker.on_change('value', update_threshold)
    
    doc.add_root(column(file_picker, p))

    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

In [13]:
show(bkapp, notebook_url="http://localhost:7888")

In [14]:
thres = 50
p = figure(title="Maximum difference in assignment proportions at different thresholds", 
           plot_height=600, plot_width=1000, y_range=(-1,105), x_range=[0, 200],
           background_fill_color='#efefef')
p.xaxis.axis_label = 'No. participants'
p.xaxis.axis_label = '% of trials with Maximum difference in assignment proportions >= threshold'
pdf_data_to_display = get_data_to_display('trial_level_agg_stats.csv', 'max_asgmt_prop_diff_ge_{0}% (%) ({1})', thres)
trial_all_plot_source = bp.ColumnDataSource(pdf_data_to_display)
lst_y_col = [dct_simulation_labels[simulation_label] for simulation_label in dct_simulation_labels.keys()]
lst_line_plots = []
p.add_layout(Legend(), 'right')
colr = lst_colors[1]
col = lst_y_col[1]
for (colr, leg) in zip(lst_colors, lst_y_col):
    p.line('n_participants', leg, color=colr, legend_label=leg, source=trial_all_plot_source, #name='wave',
            nonselection_alpha=0.4, selection_alpha=1)


In [15]:
def update(thres=50):
    pdf_data_to_display = get_data_to_display('trial_level_agg_stats.csv', 'max_asgmt_prop_diff_ge_{0}% (%) ({1})', thres)
    r.data_source.data[col] = pdf_data_to_display[col]
#     trial_all_plot_source.data.update(bp.ColumnDataSource(pdf_data_to_display).data)
    push_notebook()

In [16]:
show(p, notebook_handle=True)

In [22]:
interact(update, thres=list(range(10, 110, 10)))

interactive(children=(Dropdown(description='thres', index=4, options=(10, 20, 30, 40, 50, 60, 70, 80, 90, 100)…

<function __main__.update(thres=50)>